# Distance de Levenshtein

**Objectifs :**  
- Définir une notion de distance distance mesurant à quel point deux mots sont similaires
- Étant donné un mot et un lexique, déterminer quels mots du lexique sont les plus proches du mot fourni

**Opérations :**  
- *Remplacement* d'une lettre par une autre
- *Insertion* d'une lettre
- *Suppression* d'une lettre

**Définition (distance de Levenshtein) :**  
La distance $d(u, v)$ entre deux mots $u$ et $v$ est le plus petit entier $k$ tel que $v$ peut être obtenu à partir de $v$ par une série de $k$ insertions, remplacements ou suppressions.

**Exemples :**
- $d(\text{chateau}, \text{bateau}) = 2$  
- $d(\text{chat}, \text{batte}) = 4$  

## Calcul de la distance entre deux mots

**Observations :**
- Si les deux mots dont on cherche la distance commencent par la même lettre, il n'est jamais avantageux de faire une insertion ou une suppression sur cette lettre (ni évidemment un remplacement).  
- Si les deux mots commencent par des lettres différentes, on ne peut pas savoir quelle opération il est préférable d'appliquer sur ce couple de lettres sans regarder la suite (exemples : chat et hat, chat et achat, plat et flat).

**Définition inductive :**  
- La distance entre un mot vide et un autre mot est la longueur de cet autre mot.
- Soient $x$ et $y$ des lettres, $u$ et $v$ des mots, la distance entre les mots $xu$ et $yv$ est obtenue en considérant le calcul suivant : 
  $$
  d(xu, yv) = 
  \begin{cases}
  1 + \min \left\{ d(u, yv), d(u, v), d(xu, v) \right\} & \text{ si } x \neq y \\
  d(u, v) & \text{ sinon}
  \end{cases}
  $$

On peut prouver (mais pas aujourd'hui) que ce calcul détermine bien le plus petit nombre de transformations permettant de passer du premier mot au second.

### Implémentation récursive

In [5]:
def distance_rec(un_mot, autre_mot):
    if len(un_mot) == 0:
        return len(autre_mot)
    if len(autre_mot) == 0:
        return len(un_mot)
    if un_mot[0] == autre_mot[0]:
        return distance_rec(un_mot[1:], autre_mot[1:])
    d_rempl = distance_rec(un_mot[1:], autre_mot[1:])
    d_ins =   distance_rec(un_mot,     autre_mot[1:])
    d_sup =   distance_rec(un_mot[1:], autre_mot)
    return 1 + min(d_rempl, d_ins, d_sup)

In [6]:
distance_rec("", "chat")

4

In [7]:
distance_rec("chat", "")

4

In [8]:
distance_rec("chateau", "bateau")

2

In [9]:
distance_rec("chat", "batte")

4

In [10]:
distance_rec("anticonstitutionnellement", "c'est n'importe quoi")

KeyboardInterrupt: 

### Implémentation itérative

Une autre approche consiste à construire ligne par ligne un tableau indiquant la distance entre chaque préfixe des deux mots. Une nouvelle case peut être remplie en regardant le contenu des trois cases en haut, à gauche et en diagonale (vers le haut et la gauche).

Ce type d'approche qui consiste à construire un résultat en s'appuyant sur un tableau de résultats à des sous-problèmes est souvent appelé *programmation dynamique*.

**Exemple :**

|       | **-** | **c** | **h** | **a** | **t** | **e** | **a** | **u** |
|-------|-------|-------|-------|-------|-------|-------|-------|-------|
| **-** |   0   |   1   |   2   |   3   |   4   |   5   |   6   |   7   |
| **b** |   1   |       |       |       |       |       |       |       |
| **a** |   2   |       |       |       |       |       |       |       |
| **t** |   3   |       |       |       |       |       |       |       |
| **e** |   4   |       |       |       |       |       |       |       |
| **a** |   5   |       |       |       |       |       |       |       |
| **u** |   6   |       |       |       |       |       |       |       |


**Exemple :**

|       | **-** | **s** | **a** | **b** | **r** | **e** |
|-------|-------|-------|-------|-------|-------|-------|
| **-** |   0   |   1   |   2   |   3   |   4   |   5   |
| **a** |   1   |       |       |       |       |       |
| **r** |   2   |       |       |       |       |       |
| **b** |   3   |       |       |       |       |       |
| **r** |   4   |       |       |       |       |       |
| **e** |   5   |       |       |       |       |       |


In [25]:
def distance_iter(un_mot, autre_mot):
    tableau = []
    
    # première ligne
    ligne = list(range(len(un_mot)+1))
    tableau.append(ligne)

    # lignes suivantes
    for i in range(len(autre_mot)):
        ligne = [i+1]
        tableau.append(ligne)
        for j in range(len(un_mot)):
            if un_mot[j] == autre_mot[i]:
                ligne.append(tableau[i][j])
            else:
                # les deux prochaines lettres sont différentes
                d_rempl = tableau[i][j]
                d_ins = tableau[i][j+1]
                d_sup = tableau[i+1][j]
                ligne.append(1 + min(d_rempl, d_ins, d_sup))
    return tableau[-1][-1]

In [26]:
distance_iter("", "chat")

4

In [27]:
distance_iter("chat", "")

4

In [28]:
distance_iter("chateau", "bateau")

2

In [29]:
distance_iter("arbre", "sabre")

2

In [30]:
distance_iter("chat", "batte")

4

In [31]:
distance_iter("anticonstitutionnellement", "c'est n'importe quoi")

21

On peut améliorer un peu la consommation de mémoire de cette fonction en ne mémorisant que la ligne en cours et la ligne précédente du tableau :

In [ ]:
def distance_iter_opti(un_mot, autre_mot):
    ...

In [ ]:
distance_iter_opti("", "chat")

In [ ]:
distance_iter_opti("chat", "")

In [ ]:
distance_iter_opti("chateau", "bateau")

In [ ]:
distance_iter_opti("chat", "batte")

In [ ]:
distance_iter_opti("anticonstitutionnellement", "c'est n'importe quoi")

### Implémentation récursive avec mémoïsation

La version récursive est plus simple, mais elle est beaucoup plus lente... Pour la rendre plus efficace on peut utiliser une astuce appelée "mémoisation" : on conserve dans un dictionnaire tous les résultats déjà connus, et avant de faire le moindre calcul, on vérifie dans le dictionnaire si la distance qu'on cherche n'a pas déjà été calculée.

In [ ]:
def distance_rec_opti(un_mot, autre_mot, memo=None):
    ...

In [ ]:
distance_rec_opti("", "chat")

In [ ]:
distance_rec_opti("chat", "")

In [ ]:
distance_rec_opti("chateau", "bateau")

In [ ]:
distance_rec_opti("chat", "batte")

In [ ]:
distance_rec_opti("anticonstitutionnellement", "c'est n'importe quoi")

### Une petite comparaison

In [32]:
%timeit distance_iter("bougeoir", "bavoir")
# %timeit distance_iter_opti("bougeoir", "bavoir")
%timeit distance_rec("bougeoir", "bavoir")
# %timeit distance_rec_opti("bougeoir", "bavoir")

15 μs ± 156 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
1.14 ms ± 14 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Quel intérêt à utiliser malgré tout la version mémoïsée (si on ne s'inquiète pas trop de la mémoire) ?

In [ ]:
%%timeit
distance_rec_opti("recevoir", "percevra")
distance_rec_opti("décevoir", "percevra")
distance_rec_opti("tellement", "feulement")
distance_rec_opti("feulement", "tellement")

In [ ]:
%%timeit
memo = {}
distance_rec_opti("recevoir", "percevra", memo)
distance_rec_opti("décevoir", "percevra", memo)
distance_rec_opti("tellement", "feulement", memo)
distance_rec_opti("feulement", "tellement", memo)

## Correction othographique

Étant donné un mot `mot` et une liste de mots `lexique`, il suffit de chercher tous les mots à une distance minimale de `mot` dans `lexique` pour obtenir une liste de candidats à la correction (et éventuellement le mot lui même s'il apparaît dans le lexique !).

**Variantes possibles :** renvoyer tous les mots à une distance inférieure à un certain seuil, ou seulement les mots de distance minimale (s'ils sont en-dessous du seuil).

In [ ]:
def corrections(mot, lexique, seuil=float("+inf"), min_seulement=True):
    ...

In [ ]:
for i in range(6):
    print(i, corrections("blancha", ["balance", "balança", "blanc", "calancha", "calancherai", "balancerai"], i, False))

## Pour aller plus loin

- Une structure de données (potentiellement) plus efficace : les [arbres BK](https://en.wikipedia.org/wiki/BK-tree).
- Une distance qui prend en compte les erreurs dans l'ordre des lettres : la distance de [Damerau-Levenshtein](https://fr.wikipedia.org/wiki/Distance_de_Damerau-Levenshtein).
- Comment tenir compte des informations du corpus pour améliorer la suggestion de correction ?
- Comment tenir compte de la notion de seuil pour optimiser les calculs de distance ?